In [ ]:
import pandas as pd
import numpy as np

# -----------------------------
# 1) Load data
# -----------------------------
# Update file path to your raw CSV location
path = "car_prices.csv"  
df = pd.read_csv(path)

print(df.shape)
df.head()

# -----------------------------
# 2) Standardize column names (optional but recommended)
# -----------------------------
df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]

# If your dataset uses different names, map them here.
# Example mapping (adjust as needed):
rename_map = {
    "saledate": "sale_date",
    "sellingprice": "selling_price",
    "mmr": "mmr_price",
    "year": "vehicle_year",
    "body": "body_type",
    "state": "state_code"
}
df = df.rename(columns={k:v for k,v in rename_map.items() if k in df.columns})

# Quick check
missing_expected = [c for c in ["sale_date","selling_price","mmr_price","make","model"] if c not in df.columns]
print("Missing expected columns:", missing_expected)

# -----------------------------
# 3) Type conversions
# -----------------------------
# Sale date parsing: try automatic parse; if it fails, specify format.
df["sale_date"] = pd.to_datetime(df["sale_date"], errors="coerce")

# Numeric conversions
for col in ["selling_price", "mmr_price", "odometer", "vehicle_year", "condition"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Trim key text columns
for col in ["make","model","trim","body_type","transmission","seller","state_code"]:
    if col in df.columns:
        df[col] = df[col].astype("string").str.strip()

# -----------------------------
# 4) Basic cleaning rules
# -----------------------------
before = len(df)

df = df[df["sale_date"].notna()]
df = df[df["selling_price"].notna() & (df["selling_price"] > 0)]

if "vehicle_year" in df.columns:
    current_year = pd.Timestamp.today().year
    df = df[(df["vehicle_year"].isna()) | ((df["vehicle_year"] >= 1980) & (df["vehicle_year"] <= current_year + 1))]

if "odometer" in df.columns:
    df = df[(df["odometer"].isna()) | ((df["odometer"] >= 0) & (df["odometer"] <= 500000))]

after = len(df)
print("Rows before:", before, "after:", after, "dropped:", before-after)

# -----------------------------
# 5) Feature engineering (ETL output for dashboard)
# -----------------------------
df["price_vs_mmr"] = df["selling_price"] - df["mmr_price"]

df["price_vs_mmr_pct"] = np.where(
    (df["mmr_price"].isna()) | (df["mmr_price"] == 0),
    np.nan,
    (df["selling_price"] - df["mmr_price"]) / df["mmr_price"]
)

# Pricing status (match SQL thresholds)
df["pricing_status"] = np.select(
    [
        (df["mmr_price"].isna()) | (df["mmr_price"] == 0),
        df["selling_price"] < df["mmr_price"] * 0.98,
        df["selling_price"] > df["mmr_price"] * 1.02,
    ],
    ["Unknown", "Underpriced", "Overpriced"],
    default="Fair"
)

# Time fields for Power BI
df["sale_year"] = df["sale_date"].dt.year
df["sale_month"] = df["sale_date"].dt.month
df["sale_quarter"] = df["sale_date"].dt.quarter
df["sale_month_name"] = df["sale_date"].dt.month_name()

# Optional: normalize body_type categories (example)
if "body_type" in df.columns:
    df["body_type"] = df["body_type"].str.title()

# -----------------------------
# 6) QA checks (quick sanity)
# -----------------------------
print(df[["selling_price","mmr_price","price_vs_mmr","price_vs_mmr_pct"]].describe())

print("Pricing status distribution:")
print(df["pricing_status"].value_counts(dropna=False))

# -----------------------------
# 7) Export clean datasets
# -----------------------------
# Option A: one clean dataset for Power BI
out_model = "../data/processed/car_sales_clean.csv"
df.to_csv(out_model, index=False)
print("Saved:", out_model)

# Option B: optional star-schema exports (if you prefer importing dim/fact to Power BI)
# Create dimension tables
dim_date = (
    df[["sale_date","sale_year","sale_month","sale_quarter","sale_month_name"]]
    .drop_duplicates()
    .assign(date_key=lambda x: x["sale_date"].dt.date)
    .drop(columns=["sale_date"])
)

dim_vehicle_cols = [c for c in ["vehicle_year","make","model","trim","body_type","transmission"] if c in df.columns]
dim_vehicle = df[dim_vehicle_cols].drop_duplicates().reset_index(drop=True)
dim_vehicle["vehicle_key"] = dim_vehicle.index + 1

dim_seller = df[["seller"]].fillna("Unknown").drop_duplicates().reset_index(drop=True)
dim_seller["seller_key"] = dim_seller.index + 1

dim_location = df[["state_code"]].fillna("Unknown").drop_duplicates().reset_index(drop=True)
dim_location["location_key"] = dim_location.index + 1

# Build fact table keys by merging
fact = df.copy()
fact["date_key"] = fact["sale_date"].dt.date

fact = fact.merge(dim_vehicle, on=dim_vehicle_cols, how="left")
fact = fact.merge(dim_seller, on=["seller"], how="left")
fact = fact.merge(dim_location, on=["state_code"], how="left")

fact_cols = [
    "date_key","vehicle_key","seller_key","location_key",
    "condition","odometer","selling_price","mmr_price",
    "price_vs_mmr","price_vs_mmr_pct","pricing_status"
]
fact_cols = [c for c in fact_cols if c in fact.columns]
fact_sales = fact[fact_cols]

# Save star schema
dim_date.to_csv("../data/processed/dim_date.csv", index=False)
dim_vehicle.to_csv("../data/processed/dim_vehicle.csv", index=False)
dim_seller.to_csv("../data/processed/dim_seller.csv", index=False)
dim_location.to_csv("../data/processed/dim_location.csv", index=False)
fact_sales.to_csv("../data/processed/fact_sales.csv", index=False)

print("Saved star schema files to ../data/processed/")
